# gp2Scale 
gp2Scale is a special setting in fvgp that combines non-stationary, compactly-supported kernels, HPC distributed computing, and sparse linear algebra to allow scale-up of exact GPs to millions of data points. Here we run a moderately-sized GP, just because we assume you might run this locally.

I hope it is clear how cool it is what is happening here. If you have a dask client that points to a remote cluster with 500 GPUs, you will distribute the covariance matrix computation across those. The full matrix is sparse will be fast to work with in downstream operations. The algorithm only makes use of naturally-occuring sparsity, so the result is exact in contrast to Vecchia or inducing-point methods.

In [ ]:
##first install the newest version of fvgp
#!pip install fvgp==4.0.0

## Setup

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from fvgp import GP
from dask.distributed import Client
%load_ext autoreload
%autoreload 2

client = Client() ##this is the client you can make locally like this or 
#your HPC team can provide a script to get it. We included an example to get gp2Scale going
#on Perlmutter


#It's good practice to make sure to wait for all the workers to be ready
client.wait_for_workers(4)

## Preparing the data and some other inputs

In [2]:
def f1(x):
    return ((np.sin(5. * x) + np.cos(10. * x) + (2.* (x-0.4)**2) * np.cos(100. * x)))

input_dim = 1
N = 10000
x_data = np.random.rand(N,input_dim)
y_data = f1(x_data)
hps_n = 2

hps_bounds = np.array([[0.1,10.],      ##signal var of Wendland kernel
                       [0.001,0.02]])  ##length scale for Wendland kernel
        

In [ ]:
init_hps = np.random.uniform(size = len(hps_bounds), low = hps_bounds[:,0], high = hps_bounds[:,1])

my_gp2S = GP(1, x_data,y_data,init_hps,  #compute_device = 'gpu', #you can use gpus here
            gp2Scale = True, gp2Scale_batch_size= 1000, gp2Scale_dask_client = client, info = True
            )

my_gp2S.train(hps_bounds, max_iter = 10)



We have  3  compute workers ready to go.
Actor on tcp://127.0.0.1:34877
Scheduler Address:  tcp://127.0.0.1:46797
    submitted batch. i: 0 1000    j: 0 1000 to worker  tcp://127.0.0.1:42561  Future:  kernel_function-706d1d19d6bbc87bf0cba2747fb1efc0
    current time stamp:  0.0023059844970703125  percent finished:  0.0

    submitted batch. i: 0 1000    j: 1000 2000 to worker  tcp://127.0.0.1:38885  Future:  kernel_function-53281a2d45a0f4255eecb7b541b2fd23
    current time stamp:  0.0045070648193359375  percent finished:  0.01818181818181818

    submitted batch. i: 0 1000    j: 2000 3000 to worker  tcp://127.0.0.1:39237  Future:  kernel_function-8a6b44047110fd9c3f7171fa4f7bc2f5
    current time stamp:  0.0061626434326171875  percent finished:  0.03636363636363636

    submitted batch. i: 0 1000    j: 3000 4000 to worker  tcp://127.0.0.1:42561  Future:  kernel_function-1e23c7043581d54405dac60c2caab8ad
    current time stamp:  0.7095541954040527  percent finished:  0.05454545454545454



    current time stamp:  3.3682761192321777  percent finished:  0.6545454545454545

    submitted batch. i: 4000 5000    j: 7000 8000 to worker  tcp://127.0.0.1:39237  Future:  kernel_function-dcbe80c0a19fee717b82e5e85d978866
    current time stamp:  3.371535539627075  percent finished:  0.6727272727272727

    submitted batch. i: 4000 5000    j: 8000 9000 to worker  tcp://127.0.0.1:42561  Future:  kernel_function-a847e98fbd8b1aa4103ab9f921ddb197
    current time stamp:  3.477268695831299  percent finished:  0.6909090909090909

    submitted batch. i: 4000 5000    j: 9000 10000 to worker  tcp://127.0.0.1:38885  Future:  kernel_function-f1e41679efbd647d19851449ed629055
    current time stamp:  3.584740161895752  percent finished:  0.7090909090909091

    submitted batch. i: 5000 6000    j: 5000 6000 to worker  tcp://127.0.0.1:39237  Future:  kernel_function-96c81b813f4dfe06c7dccbdd678a4b19
    current time stamp:  3.586576223373413  percent finished:  0.7272727272727273

    submitted b

Collected Future  kernel_function-e682d267fd907f2ddf77465feabcbb7e  has finished its work in 0.07158017158508301  seconds. time stamp:  5.106107234954834
Collected Future  kernel_function-25b5f6e21a26034d1f5f8ce1d8ba1161  has finished its work in 0.05093050003051758  seconds. time stamp:  5.117544174194336
Collected Future  kernel_function-8a6b44047110fd9c3f7171fa4f7bc2f5  has finished its work in 0.04666924476623535  seconds. time stamp:  5.1271326541900635
Collected Future  kernel_function-1600870ae28b4fe9f2cdf33ca55fe61b  has finished its work in 0.04598236083984375  seconds. time stamp:  5.138045310974121
Collected Future  kernel_function-8ae7b5d760ac6953abd32155fc1b368c  has finished its work in 0.058075904846191406  seconds. time stamp:  5.149068117141724
Collected Future  kernel_function-69f8f7878fc61744dd4c321b143e2bd7  has finished its work in 0.0472567081451416  seconds. time stamp:  5.160839557647705
Collected Future  kernel_function-53faf04e9ed40ca3b54b6ee7233623e5  has fin

In [ ]:
x_pred = np.linspace(0,1,100) ##for big GPs, this is usually not a good idea, but in 1d, we can still do it
                              ##It's better to do predicitons only for a handful of points.

mean1 = my_gp2S.posterior_mean(x_pred.reshape(-1,1))["f(x)"]
var1 =  my_gp2S.posterior_covariance(x_pred.reshape(-1,1), variance_only=False)["v(x)"]

print(my_gp2S.hyperparameters)

plt.figure(figsize = (16,10))
plt.plot(x_pred,mean1, label = "posterior mean", linewidth = 4)
plt.plot(x_pred,f1(x_pred), label = "latent function", linewidth = 4)
plt.fill_between(x_pred, mean1 - 3. * np.sqrt(var1), mean1 + 3. * np.sqrt(var1), alpha = 0.5, color = "grey", label = "var")
plt.scatter(x_data,y_data, color = 'black')